In [1]:
import pandas as pd
import os
from google.colab import drive 

In [2]:
DRIVE_PATH = "gdrive/MyDrive/DataMining_project/data/"
OUTPUT_DIR = "gdrive/MyDrive/temp_outputs_dm_project"

EXP_SNORNA = DRIVE_PATH + "data_expansion_input/exp_snorna.txt"
# EXP_DATASET = DRIVE_PATH + "data_expansion_input/filtered_dataset.csv"
HOST_LINKS = DRIVE_PATH + "data_sno_canonical/gene_host_link.txt"

NET_INT_PATH = DRIVE_PATH + "data_parsed_networks/networks"
MERGED_INT_PATH = DRIVE_PATH + "data_parsed_networks/single_expansions"
INPUT_DIRS = [NET_INT_PATH, MERGED_INT_PATH]

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load the files

In [6]:
# this is a dictionary that contains {key : sno_ensg_id, val : host_ensg_id}
gene_host_map = eval((open(HOST_LINKS, "r")).read())

exp_snoRNAs = pd.read_csv(EXP_SNORNA, sep='\n', header=None)
exp_snoRNAs = (exp_snoRNAs[0]).to_list()

read each file and get a number of hosts with their frel

In [7]:
columns_names = ["sno_id", "host_id", "frel"]

results = {}

Look for host - sno intreraction link in the networks

In [8]:
for dir in INPUT_DIRS: 
  # take each of the files inside (ribo, sno, ribo+sno)
    for file in os.listdir(dir):
        complete_file_path = os.path.join(dir, file)
        print(complete_file_path)

        # checking if it is a file
        if os.path.isfile(complete_file_path):
          
          output_df = pd.DataFrame(columns=columns_names)
          network = pd.read_csv(complete_file_path, index_col=0)

          for index, row in network.iterrows():
            # check if it is a sno
            if str(row.loc["x"]) in exp_snoRNAs:
              # check if the other gene is its host
              if row.loc["y"] == gene_host_map[row.loc["x"]]:
                new_row = {"sno_id" : row.loc["x"], "host_id" : row.loc["y"], "frel" : row.loc["frel"]}
                output_df = output_df.append(new_row, ignore_index=True)
                
            # check if it is a sno
            if row.loc["y"] in exp_snoRNAs:
              # check if the other gene is its host
              if row.loc["x"] == gene_host_map[row.loc["y"]]:
                new_row = {"sno_id" : row.loc["y"], "host_id" : row.loc["x"], "frel" : row.loc["frel"]}
                output_df = output_df.append(new_row, ignore_index=True)
          
          results[file] = output_df


gdrive/MyDrive/DataMining_project/data/data_parsed_networks/networks/ribosomal_net_0.csv
gdrive/MyDrive/DataMining_project/data/data_parsed_networks/networks/snorna_net_0.csv
gdrive/MyDrive/DataMining_project/data/data_parsed_networks/networks/snorna_ribo_net_0.csv
gdrive/MyDrive/DataMining_project/data/data_parsed_networks/single_expansions/merged_interactions_files_ribo.csv
gdrive/MyDrive/DataMining_project/data/data_parsed_networks/single_expansions/merged_interactions_files_sno.csv
gdrive/MyDrive/DataMining_project/data/data_parsed_networks/single_expansions/merged_interactions_files_sno_ribo.csv


In [9]:
for key, value in results.items():
  value.to_csv(OUTPUT_DIR + "/" + key.split(".")[0] + "_host_interactions.csv" )

Check for thersholded interactions

In [10]:
thresholds = [0, 0.05, 0.5, 0.9]
thresholds_columns = ["network", 0, 0.05, 0.5, 0.9]

thresholds_comparison = pd.DataFrame(columns=thresholds_columns)

In [12]:
for file_name, df in results.items():
  valid_per_thresholds = {}

  valid_per_thresholds["network"] = file_name

  for threshold in thresholds:
    valid_per_thresholds[threshold] = 0
    
    for index, row in df.iterrows():

      if row.loc["frel"] > threshold:

        valid_per_thresholds[threshold] =  valid_per_thresholds[threshold] + 1

  thresholds_comparison = thresholds_comparison.append(valid_per_thresholds, ignore_index = True)

Tabella num per threshold \\
per ciascuno dei due sno ENSG00000200959, ENSG00000202048 tabella 

In [ ]:
thresholds_comparison

cosa che mi ha chiesto stefano, non so bene cosa sia

Vittoria se ti do una matrice con sno sulle righe, tutti i geni del dataset filtrato sulle colonne, quanto ci metti a recuperare per ogni sno la cella (correlation coefficient) dell'host, e fare la stessa cosa che hai fatto per la freq relativa

In [ ]:
load_matrix = pd.read_csv("gdrive/MyDrive/DataMining_project/data/data_WGCNA/similarity_mat.csv", index_col=0, header = None)

values_per_thresholds = {}
thresholds = [0, 0.05, 0.5, 0.9]

list_genes = list(load_matrix.iloc[0]) 
list_genes

In [27]:
load_matrix = pd.read_csv("gdrive/MyDrive/DataMining_project/data/data_WGCNA/similarity_mat.csv", index_col=0)


In [32]:
for threshold in thresholds:
    values_per_thresholds[threshold] = 0

    for sno in exp_snoRNAs:

        if gene_host_map[sno] in list_genes and load_matrix.loc[sno, gene_host_map[sno]] > threshold:
            values_per_thresholds[threshold] += 1

            if threshold >= 0.5:
              print("sno:", sno, "\thost:", gene_host_map[sno], "\tvalue: ", load_matrix.loc[sno, gene_host_map[sno]])

values_per_thresholds

sno: ENSG00000200503 	host: ENSG00000224078 	value:  0.520960320836938
sno: ENSG00000206633 	host: ENSG00000115758 	value:  0.563915180096889
sno: ENSG00000270141 	host: ENSG00000270141 	value:  1.0
sno: ENSG00000270141 	host: ENSG00000270141 	value:  1.0


{0: 339, 0.05: 86, 0.5: 3, 0.9: 1}